<h1 style="color:DarkTurquoise"> CrypTen Introduction / First Attempt </h1> 

This file contains code and notes for running the first implementation of CrypTen that I have tried. 


**NOTE**: once initial attempt is complete, please move this back into the crypten_case_studies folder within the driverdrowsiness application folder

In [2]:
from models.compactcnn import CompactCNN
import torch
import crypten
import crypten.mpc as mpc 
import crypten.communicator as comm 

In [4]:
# define the parties involved
ALICE = 0
BOB = 1

In [3]:
# separate the labels and the data points themselves into separate ".pth" files to be compatible with CrypTen's system

labels = torch.load("")

In [ ]:
def compute_accuracy(output, labels):
    # need to convert this to cpu or numpy? experiment once multiprocess decorator begins to work properly
    pred = output.argmax(axis=-1)
    correct = pred.eq(labels)
    correct_count = correct.sum(0, keepdim=True).float()
    accuracy = correct_count.mul_(100.0 / output.size(0))
    return accuracy

@mpc.run_multiprocess(world_size=2)
def evaluate_model():
    # load pre-trained model to Alice
    model = crypten.load_from_party("pretrained/sub9/model.pth", src=ALICE)

    # encrypt the model from Alice
    dummy_input = torch.empty((1,1,384))
    private_model = crypten.nn.from_pytorch(model, dummy_input)
    private_model.encrypt(src=ALICE)

    # load data to Bob
    features = crypten.load_from_party("", src=BOB)
    # is this step necessary? try it out once the multiprocess thing works properly
    flattened_features = features.flatten(start_dim=1)

    # classify the encrypted data
    private_model.eval()
    output_enc = private_model(flattened_features)

    # compute the accuracy
    output = output_enc.gen_plain_text()
    accuracy = compute_accuracy(output, labels)
    crypten.print("\tAccuracy: {0:.4f}".format(accuracy.item()))